In [ ]:
# Importing Necessary Libraries
import os
import nltk
import math
import random
from collections import Counter
from collections import defaultdict


In [ ]:
# Tokenizer to keep only Alpha-Numeric Terms  [a-zA-Z0-9_]
tokenizer = nltk.RegexpTokenizer(r"\w+")

global corpus
corpus = {}

global index
index = defaultdict(list)

def PreProcess(data):
    data = ' '.join(tokenizer.tokenize(data))
    data = data.lower()
    return data